# Ce se întamplă în cazul clasificarii binare daca se modifică pragul de decizie din 0.5 în alte valori. Cum se poate aprecia calitatea clasificatorului pentru diferite valori ale pragului?

#### Raspuns: Daca pragul este mai mare (de exemplu 0.8) atunci sansele ca algoritmul sa prezica corect label-ul pozitiv sunt mai mici. Aceasta situatie poate conduce de exemplu la clasificarea unor pacienti bolnavi ca fiind sanatosi, ceea ce nu este de dorit. Daca pragul este mai mic, sansele ca cei bolnavi sa fie clasificati corect sunt mai mari. (nu este atat de grav daca pacientii sanatosi sunt considerati bolnavi) 

# Rezolvarea unei probleme de regresie/clasificare prin: folosirea validarii încrucișate (K-fold cross validation) - pb cu happiness dupa pib

In [13]:
import csv
import os
import matplotlib.pyplot as plt
import numpy as np 
from sklearn import linear_model
import pandas as pd 
from sklearn.metrics import mean_squared_error


#Ce îi poate face pe oameni fericiți? - dupa PIB
def readData(dataPath: str):
    df = pd.read_csv(dataPath, delimiter=',', header='infer')
    df = df.dropna()
    return df

#split data frame in k sets
def splitDataInKSets(dataFrame, k):
    size = dataFrame.shape[0]
    arr = np.array_split(range(size),k) 
    input = [[dataFrame["Economy..GDP.per.Capita."].iloc[i] for i in index] for index in arr]
    output = [[dataFrame["Happiness.Score"].iloc[i] for i in index] for index in arr]
    return input, output

def getErrors(computed_output, validation_output):
    computedError = mean_squared_error(validation_output, computed_output)
    return computedError

def trainRegressor(regressor, dataFrame, k):
    errors = []
    input, output = splitDataInKSets(dataFrame, k)
    for i in range(0, k):
        validationInputSet = input[i]
        validationOutputSet = output[i]
        trainingInputSet = []
        trainingOutputSet = []
        for j in range(0, k):
            if j != i:
                trainingInputSet += input[j]
                trainingOutputSet += output[j]
        regressor.partial_fit([[trainingInputSet[ind]] for ind in range(0, len(trainingInputSet))], trainingOutputSet)
        computed_output = regressor.predict([[validationInputSet[ind]] for ind in range(0, len(validationInputSet))])
        errors.append(getErrors(computed_output, validationOutputSet))
    return errors

In [25]:
dataFrame = readData("2017.csv")
regressor = linear_model.SGDRegressor()
errors = trainRegressor(regressor, dataFrame, 6)
print(errors)
overallError = sum(errors) / len(errors)
print("Overall error = ", overallError)

[10.380478165717864, 2.343952974107564, 0.4667713607167022, 0.5148440436904708, 0.5450812913388456, 0.3992505269318201]
Overall error =  2.4417297270838776
